In [ ]:
#Sever Link for the website
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
from flask import Flask, render_template, request, jsonify
import re
import torch
import pickle
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, pipeline
from peft import PeftModel, PeftConfig
from huggingface_hub import login

In [ ]:
#Place your Hugging Face Access Token Here
login(token = "HF_TOKEN")

In [ ]:
#Index.html file path.
app = Flask(__name__, template_folder="/content/templates")

In [ ]:
with app.app_context():

    global response_model, response_tokenizer, crisis_model, crisis_tokenizer, emotion_classifier, emotion_classifier_tokenizer, emotion_classifier_model


    # Load response model and tokenizer
    peft_model_id = "Prashanth74/Gemma_2b_Response_Generation_Model_MentalHealth_Chatbot"
    peft_config = PeftConfig.from_pretrained(peft_model_id)
    response_tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path)

    new_tokens = ["<extra_token_1>", "<extra_token_2>"]
    response_tokenizer.add_tokens(new_tokens)
    #print("Tokenizer size after adding tokens:", len(response_tokenizer))  # Should now be 256002

    base_model = AutoModelForCausalLM.from_pretrained(
        peft_config.base_model_name_or_path,
        device_map="auto"
    )

    base_model.resize_token_embeddings(len(response_tokenizer))
    response_model = PeftModel.from_pretrained(base_model, peft_model_id)
    print("Response Model and Tokenizer Loaded Sucessfully....")
    #response_model.eval()

    # Load crisis detection model
    crisis_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    crisis_model = AutoModelForSequenceClassification.from_pretrained("Prashanth74/BERT_Crisis_Detection_Model")
    print("Crisis Model and Tokenizer Loaded Sucessfully....")
    #crisis_model.eval()

    #Load emotion classifier model
    emotion_classifier_tokenizer = AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion")
    emotion_classifier_model = AutoModelForSequenceClassification.from_pretrained("Prashanth74/BERT_Emotion_Classification_Model")

    emotion_classifier = pipeline("text-classification", model=emotion_classifier_model, tokenizer=emotion_classifier_tokenizer)
    print("Emotion Classifier Model and Tokenizer Loaded Sucessfully....")
    #emotion_classifier.eval()

In [ ]:
emotion_labels = ['example_very_unclear',
 'admiration',
 'amusement',
 'anger',
 'annoyance',
 'approval',
 'caring',
 'confusion',
 'curiosity',
 'desire',
 'disappointment',
 'disapproval',
 'disgust',
 'embarrassment',
 'excitement',
 'fear',
 'gratitude',
 'grief',
 'joy',
 'love',
 'nervousness',
 'optimism',
 'pride',
 'realization',
 'relief',
 'remorse',
 'sadness',
 'surprise',
 'neutral']

In [ ]:
chat_history = []

In [ ]:
#Removing incomplete sentences from model response
def clean_response(text):
    sentences = re.findall(r'[^.]*\.', text)
    return ''.join(sentences).strip()

#Mapping the user_input to corresponding emotion label
def convert_label_to_emotion(predicted_emotion):

    label_to_emotion = {f"LABEL_{i}": emotion for i, emotion in enumerate(emotion_labels)}

    emotion_scores = {}
    for item in predicted_emotion:
        label = item['label']
        score = item['score']

        if label in label_to_emotion:
            emotion = label_to_emotion[label]
            emotion_scores[emotion] = score
        else:
            print(f"Warning: Label '{label}' not found in the label mapping.")

    return emotion_scores


#Classify the predictied emotion label to positive, negative or mixed class
def classify_emotion(emotion_scores):

    positive_emotions = ["admiration", "amusement", "approval", "caring", "curiosity", "desire", "excitement", "gratitude", "joy", "love", "optimism", "pride", "relief"]
    negative_emotions = ["anger", "annoyance", "disappointment", "disapproval", "disgust", "embarrassment", "fear", "grief", "nervousness", "remorse", "sadness"]
    mixed_emotions = ["example_very_unclear", "confusion", "neutral", "surprise", "realization"]

    positive_score = sum(emotion_scores.get(emotion, 0) for emotion in positive_emotions)
    negative_score = sum(emotion_scores.get(emotion, 0) for emotion in negative_emotions)
    neutral_score = sum(emotion_scores.get(emotion, 0) for emotion in mixed_emotions)

    if positive_score > negative_score:
        return "positive"
    elif negative_score > positive_score:
        return "negative"
    else:
        return "mixed"

# Crisis detection
def is_crisis(text):
    inputs = crisis_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = crisis_model(**inputs)
        prediction = torch.argmax(outputs.logits, dim=-1).item()
    return prediction == 1

# Generate chatbot reply
def generate_response(user_input):

    global chat_history

    chat_history.append({"role": "user", "content": user_input})

    prompt = response_tokenizer.apply_chat_template(chat_history, tokenize=False, add_generation_prompt=True)

    inputs = response_tokenizer(prompt,
                                return_tensors='pt',
                                padding=True,
                                truncation=True,
                                max_length=1024).to(response_model.device)

    outputs = response_model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=True,
        top_k=30,
        top_p=0.85,
        temperature=0.7,
        #top_k=50,
        #top_p=0.9,
        #repetition_penalty=1.1,
        #eos_token_id=response_tokenizer.eos_token_id,
    )

    decoded = response_tokenizer.decode(outputs[0], skip_special_tokens=True)


    if "model" in decoded:
        reply = decoded.split("model")[-1].strip()
    else:
        reply = decoded

    chat_history.append({"role": "assistant", "content": reply})

    return clean_response(reply)

In [ ]:
@app.route("/", methods=["GET"])
def index():
    return render_template("index.html")

@app.route("/chat", methods=["POST"])
def chat():

    global chat_history

    user_input = request.json["message"]

    if user_input.lower() == "exit":

        chat_history = []

        return jsonify({"response": "Goodbye! Take care. 💙",
                        "emotion":"neutral",
                        "end":True})

    if is_crisis(user_input):
        return jsonify({"response": "\ud83d\udcac It seems like you're going through a tough time. You're not alone. Please reach out to a crisis helpline: 988 (US National Crisis Line) or talk to someone you trust. \ud83d\udc99",
                        "emotion": "negative",
                        "end": False})

    raw_emotion = emotion_classifier(user_input)
    emotion_scores = convert_label_to_emotion(raw_emotion)
    emotion_class = classify_emotion(emotion_scores)

    response = generate_response(user_input)
    return jsonify({"response": response,
                    "emotion": emotion_class,
                    "end": False})

if __name__ == "__main__":
    app.run()